## Arquitectura de módulos del discriminamómetro

Desde este notebook se expone la arquitectura general del proyecto "Discriminamómetro", esto es con la intención de poder correr algún proceso en específico de manera rápida y sencilla.

# ETL
En el módulo ETL se encuentra: 

-El proceso de descarga recurrente que deberá estar programado para ejecutarse cada 15 minutos mediante un cron.
 
-El proceso de obtención de tweets ya sea por usuario, o hashtag/palabras específicas

In [1]:
import Discrim.etl as etl

obj_etl = etl.etl('PRUEBA')
print(obj_etl.nbr_TweetsXCorrida)

obj_etl.descarga_recurrente_tweets()

8
Entra a descarga_recurrente_tweets
str_ArchivoEnvio:  /home/jupyter-ronmoy007@gmail.co-0e856/discriminamometro/discriminamometro/scripts/json_apariencia_20200814_214627607720.txt
str_NombreArchivoEnS3:  00_pruebas/json_apariencia_20200814_214627607720.txt
Enviando el archivo a S3...
str_ArchivoEnvio:  /home/jupyter-ronmoy007@gmail.co-0e856/discriminamometro/discriminamometro/scripts/json_discapacidad_20200814_214628281254.txt
str_NombreArchivoEnS3:  00_pruebas/json_discapacidad_20200814_214628281254.txt
Enviando el archivo a S3...
str_ArchivoEnvio:  /home/jupyter-ronmoy007@gmail.co-0e856/discriminamometro/discriminamometro/scripts/json_edad_20200814_214628901932.txt
str_NombreArchivoEnS3:  00_pruebas/json_edad_20200814_214628901932.txt
Enviando el archivo a S3...
str_ArchivoEnvio:  /home/jupyter-ronmoy007@gmail.co-0e856/discriminamometro/discriminamometro/scripts/json_genero_20200814_214629529213.txt
str_NombreArchivoEnS3:  00_pruebas/json_genero_20200814_214629529213.txt
Enviando el

# Train
El "Discriminamómetro" está constituido por 2 capas, en la primer capa se evalúa mediante un modelo si un tweet discrimina o no. En caso de que un tweet sea discriminatorio, en la segunda capa se evaluarán 7 modelos (uno por cada categoría) para determinar el grado de pertenencia a cada una de ellas. Además de ello, para lograr capturar el contexto de cada palabra, se incluye una fase de generación de embeddings que servirá para entrenar los modelos

## Embeddings

In [3]:
import Discrim.Train.embeddings as emb

obj_emb = emb.Embeddings()

obj_emb.descargar_archivo_fuente()
obj_emb.transformar_dataset()
obj_emb.generar_embeddings()
obj_emb.enviar_embeddings_s3()

[nltk_data] Downloading package punkt to /home/jupyter-
[nltk_data]     ronmoy007@gmail.co-0e856/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


str_ArchivoEnvio:  ./data/word2vec_Embeddings.txt
str_NombreArchivoEnS3:  00_entrenamiento/embeddings/word2vec_Embeddings.txt
Enviando el archivo a S3...
Envío completado


# Capa1

In [5]:
! gzip ./data/word2vec_Embeddings.txt

In [6]:
! python -m spacy init-model es data --vectors-loc ./data/word2vec_Embeddings.txt.gz

✔ Successfully created model
53549it [00:09, 5552.76it/s]word2vec_Embeddings.txt.gz
✔ Loaded vectors from data/word2vec_Embeddings.txt.gz
✔ Sucessfully compiled vocab
53767 entries, 53549 vectors


In [1]:
import Discrim.Train.capa1 as cp1

obj_capa1 = cp1.capa1()
obj_capa1.descargar_tweets_entrenamiento()
obj_capa1.transformar_dataset()
obj_capa1.train()
obj_capa1.crear_pickle()
obj_capa1.mandar_pickle_s3()

[nltk_data] Downloading package punkt to /home/jupyter-
[nltk_data]     ronmoy007@gmail.co-0e856/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    2.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


grid_search.best_params_:  {'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 4}
grid_search.best_score_:  0.6472299043463667
Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    7.3s finished
/home/jupyter-ronmoy007@gmail.co-0e856/.local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


grid_search.best_params_:  {'max_depth': 7, 'max_features': 'sqrt', 'min_samples_leaf': 7, 'min_samples_split': 4, 'n_estimators': 50}
grid_search.best_score_:  0.677574089612351
Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   17.1s
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:   34.2s finished
/home/jupyter-ronmoy007@gmail.co-0e856/.local/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


grid_search.best_params_:  {'learning_rate': 0.1, 'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 7, 'min_samples_split': 16, 'n_estimators': 50}
grid_search.best_score_:  0.6936801476757846
str_ArchivoEnvio:  modelo_capa1.p
str_NombreArchivoEnS3:  01_modelos/capa1/modelo_capa1.p
Enviando el archivo a S3...
Envío completado


In [2]:
import numpy as np

npEmbeddings = obj_capa1.generar_embeddings(obj_capa1.test)

predicted = obj_capa1.best_model.predict(npEmbeddings)

np.mean(predicted == obj_capa1.test.label)

0.665

str_ArchivoEnvio:  modelo_capa1.p
str_NombreArchivoEnS3:  01_modelos/capa1/modelo_capa1.p
Enviando el archivo a S3...
Envío completado


# Capa2
En la segunda capa, se evaluarán 7 modelos

In [1]:
import Discrim.Train.Capa2.apariencia as apa

obj_apariencia = apa.Apariencia()
obj_apariencia.descargar_tweets_entrenamiento()
obj_apariencia.transformar_dataset()
obj_apariencia.pd_fuente

[nltk_data] Downloading package punkt to /home/jupyter-
[nltk_data]     ronmoy007@gmail.co-0e856/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Unnamed: 0,full_text,categoria_1-ap,categoria_2-dis,categoria_3-ed,label,categoria_5-id,categoria_6-or,categoria_7-rel
0,0,[] proximamente rd jodido el4tobate,1,0,0,0,0,0,0
1,1,jajajaja acomplejado guerito rockero ja facil ...,1,0,0,0,0,0,0
2,2,momentos cuenta contados dedos personas sincer...,1,0,0,0,0,0,0
3,3,guerito,1,0,0,0,0,0,0
4,4,restriccion preparado casos covid-19 suben 15 ...,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
495,497,parece super nefasto gente llame echando ganas...,1,0,0,1,0,0,0
496,498,super caga tipo creen super super perrisimas a...,1,0,0,1,0,0,0
497,499,mejor amlrover incluye chofer come duele mante...,1,0,0,1,0,0,0
498,500,ahhhhh quedes 1 culpa cajeros 49 cambio sufici...,1,0,0,1,0,0,0


In [10]:
import numpy as np
npEmbeddings = np.empty([0, 300])

obj_apariencia.cargar_modelo_embeddings()
for texto in obj_apariencia.pd_fuente['full_text']:

    #print(texto)

    # process a sentence using the model
    doc = obj_apariencia.nlp(texto)

    # print(doc.vector.shape)
    if doc.vector.shape[0]==300:
        npEmbeddings = np.append(npEmbeddings, [doc.vector], axis = 0)
    else:
        npAux = np.empty([1, 300])
        npEmbeddings = np.append(npEmbeddings, npAux, axis = 0)

obj_apariencia.pd_fuente['embeddings'] = list(npEmbeddings)
obj_apariencia.pd_fuente

,Unnamed: 0,full_text,categoria_1-ap,categoria_2-dis,categoria_3-ed,label,categoria_5-id,categoria_6-or,categoria_7-rel,embeddings
0,0,[] proximamente rd jodido el4tobate,1,0,0,0,0,0,0,"[-0.7870798110961914, 0.4222928285598755, 0.02..."
1,1,jajajaja acomplejado guerito rockero ja facil ...,1,0,0,0,0,0,0,"[-0.4006634056568146, 0.262923002243042, -0.17..."
2,2,momentos cuenta contados dedos personas sincer...,1,0,0,0,0,0,0,"[-0.5835394263267517, 0.38197824358940125, -0...."
3,3,guerito,1,0,0,0,0,0,0,"[-0.16571150720119476, 0.10299144685268402, -0..."
4,4,restriccion preparado casos covid-19 suben 15 ...,1,0,0,0,0,0,0,"[-0.6384806632995605, 0.39051029086112976, -0...."
...,...,...,...,...,...,...,...,...,...,...
495,497,parece super nefasto gente llame echando ganas...,1,0,0,1,0,0,0,"[-0.45719558000564575, 0.2975442111492157, -0...."
496,498,super caga tipo creen super super perrisimas a...,1,0,0,1,0,0,0,"[-0.7496370673179626, 0.43418794870376587, -0...."
497,499,mejor amlrover incluye chofer come duele mante...,1,0,0,1,0,0,0,"[-0.2823105752468109, 0.18225260078907013, -0...."
498,500,ahhhhh quedes 1 culpa cajeros 49 cambio sufici...,1,0,0,1,0,0,0,"[-0.335415780544281, 0.21464064717292786, -0.1..."


In [ ]:
obj_apariencia.pd_fuente = obj_apariencia.generar_embeddings(obj_apariencia.pd_fuente)

In [14]:
import pandas as pd
pd.DataFrame(obj_apariencia.pd_fuente['embeddings'])


,embeddings
0,"[-0.7870798110961914, 0.4222928285598755, 0.02..."
1,"[-0.4006634056568146, 0.262923002243042, -0.17..."
2,"[-0.5835394263267517, 0.38197824358940125, -0...."
3,"[-0.16571150720119476, 0.10299144685268402, -0..."
4,"[-0.6384806632995605, 0.39051029086112976, -0...."
...,...
495,"[-0.45719558000564575, 0.2975442111492157, -0...."
496,"[-0.7496370673179626, 0.43418794870376587, -0...."
497,"[-0.2823105752468109, 0.18225260078907013, -0...."
498,"[-0.335415780544281, 0.21464064717292786, -0.1..."


In [9]:
import Discrim.Train.Capa2.apariencia as apa

obj_apariencia = apa.Apariencia()
obj_apariencia.descargar_tweets_entrenamiento()
obj_apariencia.transformar_dataset()
obj_apariencia.cargar_modelo_embeddings()
obj_apariencia.pd_fuente


[nltk_data] Downloading package punkt to /home/jupyter-
[nltk_data]     ronmoy007@gmail.co-0e856/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Unnamed: 0,full_text,categoria_1-ap,categoria_2-dis,categoria_3-ed,label,categoria_5-id,categoria_6-or,categoria_7-rel
0,0,[] proximamente rd jodido el4tobate,1,0,0,0,0,0,0
1,1,jajajaja acomplejado guerito rockero ja facil ...,1,0,0,0,0,0,0
2,2,momentos cuenta contados dedos personas sincer...,1,0,0,0,0,0,0
3,3,guerito,1,0,0,0,0,0,0
4,4,restriccion preparado casos covid-19 suben 15 ...,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
495,497,parece super nefasto gente llame echando ganas...,1,0,0,1,0,0,0
496,498,super caga tipo creen super super perrisimas a...,1,0,0,1,0,0,0
497,499,mejor amlrover incluye chofer come duele mante...,1,0,0,1,0,0,0
498,500,ahhhhh quedes 1 culpa cajeros 49 cambio sufici...,1,0,0,1,0,0,0


In [10]:
obj_apariencia.pd_fuente['embeddings'] = 0
obj_apariencia.pd_fuente

,Unnamed: 0,full_text,categoria_1-ap,categoria_2-dis,categoria_3-ed,label,categoria_5-id,categoria_6-or,categoria_7-rel,embeddings
0,0,[] proximamente rd jodido el4tobate,1,0,0,0,0,0,0,0
1,1,jajajaja acomplejado guerito rockero ja facil ...,1,0,0,0,0,0,0,0
2,2,momentos cuenta contados dedos personas sincer...,1,0,0,0,0,0,0,0
3,3,guerito,1,0,0,0,0,0,0,0
4,4,restriccion preparado casos covid-19 suben 15 ...,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
495,497,parece super nefasto gente llame echando ganas...,1,0,0,1,0,0,0,0
496,498,super caga tipo creen super super perrisimas a...,1,0,0,1,0,0,0,0
497,499,mejor amlrover incluye chofer come duele mante...,1,0,0,1,0,0,0,0
498,500,ahhhhh quedes 1 culpa cajeros 49 cambio sufici...,1,0,0,1,0,0,0,0


In [13]:
%%time
import dask.dataframe as dd

#obj_apariencia.pd_fuente['embeddings'] = []
ddf = dd.from_pandas(obj_apariencia.pd_fuente, npartitions=4)

obj_apariencia.pd_fuente = ddf.map_partitions(obj_apariencia.generar_embeddings, meta=ddf).compute()

CPU times: user 1.22 s, sys: 173 ms, total: 1.39 s
Wall time: 1.32 s


In [14]:
obj_apariencia.pd_fuente

,Unnamed: 0,full_text,categoria_1-ap,categoria_2-dis,categoria_3-ed,label,categoria_5-id,categoria_6-or,categoria_7-rel,embeddings
0,0,[] proximamente rd jodido el4tobate,1,0,0,0,0,0,0,"[-0.7870798110961914, 0.4222928285598755, 0.02..."
1,1,jajajaja acomplejado guerito rockero ja facil ...,1,0,0,0,0,0,0,"[-0.4006634056568146, 0.262923002243042, -0.17..."
2,2,momentos cuenta contados dedos personas sincer...,1,0,0,0,0,0,0,"[-0.5835394263267517, 0.38197824358940125, -0...."
3,3,guerito,1,0,0,0,0,0,0,"[-0.16571150720119476, 0.10299144685268402, -0..."
4,4,restriccion preparado casos covid-19 suben 15 ...,1,0,0,0,0,0,0,"[-0.6384806632995605, 0.39051029086112976, -0...."
...,...,...,...,...,...,...,...,...,...,...
495,497,parece super nefasto gente llame echando ganas...,1,0,0,1,0,0,0,"[-0.45719558000564575, 0.2975442111492157, -0...."
496,498,super caga tipo creen super super perrisimas a...,1,0,0,1,0,0,0,"[-0.7496370673179626, 0.43418794870376587, -0...."
497,499,mejor amlrover incluye chofer come duele mante...,1,0,0,1,0,0,0,"[-0.2823105752468109, 0.18225260078907013, -0...."
498,500,ahhhhh quedes 1 culpa cajeros 49 cambio sufici...,1,0,0,1,0,0,0,"[-0.335415780544281, 0.21464064717292786, -0.1..."


In [15]:
import pandas as pd

X_train = pd.DataFrame(obj_apariencia.pd_fuente.embeddings.tolist(), index= obj_apariencia.pd_fuente.index)
X_train
#X_train = pd.DataFrame(obj_apariencia.pd_fuente['embeddings'])
#X_train

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.787080,0.422293,0.025189,-0.396396,0.044798,0.686313,-0.127823,-0.259181,0.387489,0.449993,...,-0.200189,-0.430867,-0.532225,-0.015958,0.454553,-0.014683,-0.053325,-0.253780,0.556285,-0.330296
1,-0.400663,0.262923,-0.177200,-0.091201,-0.006422,0.408766,0.028139,-0.129770,0.267855,0.183883,...,-0.043855,-0.226690,-0.172255,-0.159905,0.122269,-0.056362,-0.050374,-0.199631,0.359238,-0.178761
2,-0.583539,0.381978,-0.256741,-0.134754,-0.007903,0.596175,0.038321,-0.189095,0.390859,0.265397,...,-0.065440,-0.331244,-0.252328,-0.232985,0.177199,-0.081296,-0.071712,-0.290336,0.522698,-0.262577
3,-0.165712,0.102991,-0.052740,-0.048284,0.001366,0.163897,0.002656,-0.052233,0.102989,0.081797,...,-0.025110,-0.092753,-0.082210,-0.050729,0.063703,-0.020601,-0.021362,-0.077434,0.141606,-0.077098
4,-0.638481,0.390510,-0.173422,-0.204154,0.014884,0.626073,-0.011570,-0.204222,0.388431,0.313903,...,-0.111090,-0.362406,-0.339940,-0.175328,0.249765,-0.075696,-0.070214,-0.292182,0.525675,-0.297336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,-0.457196,0.297544,-0.195432,-0.109529,-0.005701,0.466939,0.028132,-0.149479,0.304377,0.210251,...,-0.054477,-0.258620,-0.202248,-0.176499,0.143008,-0.063933,-0.056127,-0.227325,0.407393,-0.206535
496,-0.749637,0.434188,-0.106562,-0.302493,0.023434,0.702861,-0.057687,-0.249243,0.419551,0.395729,...,-0.156200,-0.417263,-0.444285,-0.118685,0.357036,-0.053276,-0.070689,-0.297380,0.584485,-0.334625
497,-0.282311,0.182253,-0.113985,-0.072275,-0.002766,0.284686,0.013397,-0.090734,0.185090,0.131409,...,-0.035690,-0.160241,-0.128645,-0.104040,0.093458,-0.035956,-0.033185,-0.137584,0.249809,-0.125674
498,-0.335416,0.214641,-0.126006,-0.087450,0.000445,0.340284,0.013655,-0.108692,0.215910,0.158754,...,-0.045907,-0.190342,-0.158591,-0.118645,0.113331,-0.045460,-0.042307,-0.163679,0.292993,-0.155111


In [1]:
import Discrim.Train.Capa2.apariencia as apa

obj_apariencia = apa.Apariencia()
obj_apariencia.descargar_tweets_entrenamiento()
obj_apariencia.transformar_dataset()
obj_apariencia.train()
obj_apariencia.crear_pickle()
obj_apariencia.mandar_pickle_s3()

In [2]:
import Discrim.Train.Capa2.discapacidad as disc

obj_discapacidad = disc.Discapacidad()
obj_discapacidad.descargar_tweets_entrenamiento()
obj_discapacidad.transformar_dataset()
obj_discapacidad.train()
obj_discapacidad.crear_pickle()
obj_discapacidad.mandar_pickle_s3()

In [3]:
import Discrim.Train.Capa2.edad as edad

obj_edad = edad.Edad()
obj_edad.descargar_tweets_entrenamiento()
obj_edad.transformar_dataset()
obj_edad.train()
obj_edad.crear_pickle()
obj_edad.mandar_pickle_s3()

In [5]:
import Discrim.Train.Capa2.genero as gen

obj_genero = gen.Genero()
obj_genero.descargar_tweets_entrenamiento()
obj_genero.transformar_dataset()
obj_genero.train()
obj_genero.crear_pickle()
obj_genero.mandar_pickle_s3()

[nltk_data] Downloading package punkt to /home/jupyter-
[nltk_data]     ronmoy007@gmail.co-0e856/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    1.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


grid_search.best_params_:  {'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 16}
grid_search.best_score_:  0.5657705206378987
Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    5.2s finished
/home/jupyter-ronmoy007@gmail.co-0e856/.local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


grid_search.best_params_:  {'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 7, 'min_samples_split': 4, 'n_estimators': 50}
grid_search.best_score_:  0.5509621638524078
Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:   18.2s finished
/home/jupyter-ronmoy007@gmail.co-0e856/.local/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


grid_search.best_params_:  {'learning_rate': 0.1, 'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 16, 'n_estimators': 50}
grid_search.best_score_:  0.5678391963727331
str_ArchivoEnvio:  modelo_capa2_genero.p
str_NombreArchivoEnS3:  01_modelos/capa2/genero/modelo_capa2_genero.p
Enviando el archivo a S3...
Envío completado


In [4]:
import Discrim.Train.Capa2.ideologia as ideo

obj_ideologia = ideo.Ideologia()
obj_ideologia.descargar_tweets_entrenamiento()
obj_ideologia.transformar_dataset()
obj_ideologia.train()
obj_ideologia.crear_pickle()
obj_ideologia.mandar_pickle_s3()

In [5]:
import Discrim.Train.Capa2.orientacion as orien

obj_orientacion = orien.Orientacion()
obj_orientacion.descargar_tweets_entrenamiento()
obj_orientacion.transformar_dataset()
obj_orientacion.train()
obj_orientacion.crear_pickle()
obj_orientacion.mandar_pickle_s3()

In [6]:
import Discrim.Train.Capa2.religion as relig

obj_religion = relig.Religion()
obj_religion.descargar_tweets_entrenamiento()
obj_religion.transformar_dataset()
obj_religion.train()
obj_religion.crear_pickle()
obj_religion.mandar_pickle_s3()

In [6]:
obj_genero.best_model

GradientBoostingClassifier(max_features='sqrt', min_samples_leaf=3,
                           min_samples_split=16, n_estimators=50)

In [3]:
obj_genero.pd_fuente

,Unnamed: 0,full_text,categoria_1-ap,categoria_2-dis,categoria_3-ed,label,categoria_5-id,categoria_6-or,categoria_7-rel
0,0,[] proximamente rd jodido el4tobate,1,0,0,0,0,0,0
1,1,jajajaja acomplejado guerito rockero ja facil ...,1,0,0,0,0,0,0
2,2,momentos cuenta contados dedos personas sincer...,1,0,0,0,0,0,0
3,3,guerito,1,0,0,0,0,0,0
4,4,restriccion preparado casos covid-19 suben 15 ...,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
495,497,parece super nefasto gente llame echando ganas...,1,0,0,0,0,0,0
496,498,super caga tipo creen super super perrisimas a...,1,0,0,0,0,0,0
497,499,mejor amlrover incluye chofer come duele mante...,1,0,0,0,0,0,0
498,500,ahhhhh quedes 1 culpa cajeros 49 cambio sufici...,1,0,0,0,0,0,0


# Predict

Se creará una clase concentradora que tendrá instanciados todos los modelos para que únicamente sea necesario instanciar un objeto que tenga acceso a todos los clasificadores.

In [1]:
import Discrim.Predict.modelos as mod

obj_modelos = mod.Modelos()

/home/jupyter-ronmoy007@gmail.co-0e856/.local/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator DummyClassifier from version 0.22.1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/jupyter-ronmoy007@gmail.co-0e856/.local/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.22.1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/jupyter-ronmoy007@gmail.co-0e856/.local/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator GradientBoostingClassifier from version 0.22.1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [2]:
# Capa1
print('obj_modelos.discr.modelo')
print(obj_modelos.discr.modelo)

# Capa2
print('obj_modelos.apar.modelo')
print(obj_modelos.apar.modelo)

print('obj_modelos.discap.modelo')
print(obj_modelos.discap.modelo)

print('obj_modelos.edad.modelo')
print(obj_modelos.edad.modelo)

print('obj_modelos.genero.modelo')
print(obj_modelos.genero.modelo)

print('obj_modelos.ideo.modelo')
print(obj_modelos.ideo.modelo)

print('obj_modelos.orien.modelo')
print(obj_modelos.orien.modelo)

print('obj_modelos.relig.modelo')
print(obj_modelos.relig.modelo)

obj_modelos.discr.modelo
GradientBoostingClassifier(max_depth=4, max_features='sqrt', min_samples_leaf=7,
                           min_samples_split=16, n_estimators=50)
obj_modelos.apar.modelo
GradientBoostingClassifier(learning_rate=0.25, max_depth=7, max_features='sqrt',
                           min_samples_leaf=7, min_samples_split=16,
                           n_estimators=150)
obj_modelos.discap.modelo
GradientBoostingClassifier(max_depth=7, max_features='sqrt', min_samples_leaf=3,
                           min_samples_split=4)
obj_modelos.edad.modelo
GradientBoostingClassifier(learning_rate=0.75, max_depth=6, max_features='sqrt',
                           min_samples_leaf=3, min_samples_split=16)
obj_modelos.genero.modelo
GradientBoostingClassifier(max_depth=7, max_features='sqrt', min_samples_leaf=3,
                           min_samples_split=4)
obj_modelos.ideo.modelo
GradientBoostingClassifier(max_depth=7, max_features='sqrt', min_samples_leaf=3,
                    

# API

In [1]:
import Discrim.api as api

obj_api = api.API()

/home/jupyter-ronmoy007@gmail.co-0e856/.local/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator DummyClassifier from version 0.22.1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/jupyter-ronmoy007@gmail.co-0e856/.local/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.22.1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/jupyter-ronmoy007@gmail.co-0e856/.local/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator GradientBoostingClassifier from version 0.22.1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
[nltk_data] Downloading package punkt to /home/jupyter-
[nltk_data]     ronmoy007@gmail.co-0e856/nltk_data...
[nl

In [2]:
resultado = obj_api.clasificar_x_texto('Eres bien pinche fresa morra puta')
resultado

{'discriminacion': array([[0.48096646, 0.51903354]]),
 'apariencia': array([[0.52129451, 0.47870549]]),
 'discapacidad': array([[0.71725481, 0.28274519]]),
 'edad': array([[2.34046354e-04, 9.99765954e-01]]),
 'genero': array([[0.9959113, 0.0040887]]),
 'ideologia': array([[0.77009721, 0.22990279]]),
 'orientacion': array([[0.96403818, 0.03596182]]),
 'religion': array([[0.99159682, 0.00840318]])}

In [4]:
obj_api.df
obj_api.X_predict
#obj_api.obj_modelos.discr.modelo.predict(obj_api.X_predict)

array([[-0.82643127,  0.55184793, -0.38757259, -0.18697152, -0.03525492,
         0.86811399,  0.05395273, -0.22232899,  0.55013621,  0.40221825,
         0.12350072, -0.43662292, -0.1591855 ,  0.13132776, -0.29102319,
         0.03340085, -0.07259918,  0.57745284, -0.34470457, -0.61702812,
         0.20637271,  0.1138206 ,  0.36177826,  0.39817369, -0.2020102 ,
         0.53239143, -0.3481248 ,  0.00835695,  0.52556825,  0.11486737,
        -0.17779967,  0.28646976, -0.28659642, -0.25768298,  0.04271941,
        -0.86613476, -0.11046004, -0.1610668 ,  0.35799798, -0.4991625 ,
         0.1550267 , -0.35646325,  0.5832473 , -0.52410626,  0.58927453,
        -0.02650597,  0.00213802,  0.34648907,  0.60961944, -0.61191475,
         0.22794847,  0.01538521,  0.3842659 ,  0.11934608, -0.01746414,
         0.17948925,  0.21491273, -0.03372985, -0.5224582 ,  0.05562387,
        -0.68403608, -0.25745136,  0.24952456,  0.10636877,  0.06764869,
        -0.33275315, -0.07828519,  0.75855398,  0.2

In [2]:
resultado = obj_api.clasificar_x_ht('#AMLO')
resultado

{'discriminacion': array([[0.71451419, 0.28548581],
        [0.88525744, 0.11474256],
        [0.57819065, 0.42180935]]),
 'apariencia': array([[0.59880506, 0.40119494],
        [0.80345456, 0.19654544],
        [0.50186482, 0.49813518]]),
 'discapacidad': array([[0.82893627, 0.17106373],
        [0.94445969, 0.05554031],
        [0.87763891, 0.12236109]]),
 'edad': array([[0.80908939, 0.19091061],
        [0.99020284, 0.00979716],
        [0.21846075, 0.78153925]]),
 'genero': array([[0.99472059, 0.00527941],
        [0.99313143, 0.00686857],
        [0.99142236, 0.00857764]]),
 'ideologia': array([[0.87393149, 0.12606851],
        [0.85160506, 0.14839494],
        [0.84260832, 0.15739168]]),
 'orientacion': array([[0.96039805, 0.03960195],
        [0.97375216, 0.02624784],
        [0.98021912, 0.01978088]]),
 'religion': array([[0.99734279, 0.00265721],
        [0.99653868, 0.00346132],
        [0.97501664, 0.02498336]])}

In [3]:
#obj_api.list_Formateada
obj_api.df

,user_id,name,screen_name,full_text,location,embeddings
0,26922771,Panda indignado,legrandpoupon,siempre denuncio amlo ahora poder hace exactam...,"Mexico, D.F.","[-0.5659217834472656, 0.36293721199035645, -0...."
1,597898675,CEC,CirockAse,reyeseste avion amlo juntosharemoshistoria yoc...,"Guadalajara, Jalisco","[-0.4444703161716461, 0.23701530694961548, 0.0..."
2,1222911453918724096,NEBUR,NEBUR69235650,poder derecha reduce fake news decir falso apo...,,"[-0.4733160436153412, 0.3069900572299957, -0.1..."


# Dashboard

In [6]:
import Discrim.Dashboard as dash